In [ ]:
import pandas as pd
df=pd.read_xml(r"C:\Users\HP\Downloads\Orders")

import time
from collections import defaultdict
from heapq import heappush, heappop

class Order:
    def _init_(self, order_id, price, volume, operation):
        self.order_id = order_id
        self.price = price
        self.volume = volume
        self.operation = operation

class OrderBook:
    def _init_(self, book_name):
        self.book_name = book_name
        self.buy_orders = []
        self.sell_orders = []
    
    def insert_order(self, order):
        if order.operation == "BUY":
            heappush(self.buy_orders, order)
        else:
            heappush(self.sell_orders, order)
    
    def match_order(self, order):
        # Match incoming order with orders in the book
        if order.operation == "BUY":
            matched_order = self.match_buy_order(order)
        else:
            matched_order = self.match_sell_order(order)
        
        # If the incoming order is completely matched, delete it
        if order.volume == 0:
            return None
        
        # If the incoming order is partly or not matched, insert it into the book
        return order
    
    def match_buy_order(self, buy_order):
        if not self.sell_orders:
            return None
        
        # Get the highest priority sell order
        sell_order = self.sell_orders[0]
        
        # If the prices do not match, return
        if sell_order.price > buy_order.price:
            return None
        
        # Get the minimum volume between the buy and sell orders
        min_volume = min(buy_order.volume, sell_order.volume)
        
        # Reduce the volumes of the buy and sell orders by the matched volume
        buy_order.volume -= min_volume
        sell_order.volume -= min_volume
        
        # If the sell order is completely matched, remove it from the book
        if sell_order.volume == 0:
            heappop(self.sell_orders)
        
        return sell_order
    
    def match_sell_order(self, sell_order):
        if not self.buy_orders:
            return None
        
        # Get the lowest priority buy order
        buy_order = self.buy_orders[0]
        
        # If the prices do not match, return
        if buy_order.price < sell_order.price:
            return None
        
        # Get the minimum volume between the buy and sell orders
        min_volume = min(buy_order.volume, sell_order.volume)
        
        # Reduce the volumes of the buy and sell orders by the matched volume
        buy_order.volume -= min_volume
        sell_order.volume -= min_volume
    
        
        # If the buy order is completely matched, remove it from the book
        if buy_order.volume == 0:
            heappop(self.buy_orders)
        
        return buy_order